In [ ]:
# =========================
# CELDA 1: Librerías e Inicialización
# =========================
import requests  # Para hacer solicitudes HTTP
from bs4 import BeautifulSoup   # Para parsear HTML
import json              # para filtrar los datos scrapeados
import sqlite3                         # es una base de datos ligera y relacional que se almacena en un solo archivo
import time           # Para manejar retrasos entre solicitudes
import re                # Para operaciones con expresiones regulares
from urllib.parse import urljoin              # Para manejar URLs
import random                       # Para generar autores aleatorios

# Base URL para concatenar rutas
URL_BASE = "https://books.toscrape.com/"                  # URL base del sitio web a scrapear

# Lista de autores ficticios para simular datos M2M
AUTORES_FICTICIOS = [
    "Oscar Lopez", "jose perez", "juana de armas", "Ava Lunam",
    "Kaelen Rhys", "Seraphina Key", "Jaxon Bellwether", "Nadia Volkov",
    "Quinn Sparrow", "Talon Cross", "Viola Frost", "Zane O’Connell","Laura Benítez",
    "Diego Montenegro", "Sofía Rivas", "Martín López", "Camila Duarte",
    "Julián Barrios", "Valentina Coronel", "Andrés Maldonado", "Paula Sosa", "Ricardo Vázquez",
    "Elena Caballero", "Tomás Ferreira", "Gabriela Ayala", "Lucas Giménez", "Daniela Ortiz",
    "Federico Román", "María Villalba", "Gustavo Cabrera", "Nadia Paredes", "Bruno Acosta",
    "Leticia Moreno", "Rodrigo Vera", "Ana Quiñónez", "Sebastián Roldán", "Carla Medina"
]

mapeo_calificacion_texto_a_numero = {             # Mapeo de calificaciones de texto a números
    "One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5
}



In [ ]:
# =========================
# CELDA 2: Funciones de Scraping 
# =========================

def obtener_sopa_html(url):
    """Obtiene el HTML de una URL con reintentos y manejo de errores."""
    for _ in range(3):          # Intenta hasta 3 veces
        try:
            respuesta_http = requests.get(url, timeout=10)        # Realiza la solicitud HTTP y tiene un tiempo de espera de 10 segundos
            respuesta_http.raise_for_status()                      # Lanza un error si la respuesta HTTP no es exitosa
            return BeautifulSoup(respuesta_http.text, "html.parser")   #BeautifulSoup transforma el HTML en un mapa navegable, donde podemos ir directamente a los datos que nos interesa
        except requests.exceptions.RequestException as e:                     # Captura cualquier excepción relacionada con la solicitud HTTP
            print(f"Error HTTP en {url}: {e}. Reintentando...")
            time.sleep(1)              # Espera 1 segundo antes de reintentar
    raise Exception(f"No se pudo obtener el HTML de {url} después de varios intentos.")       # Si falla después de 3 intentos, lanza una excepción


def obtener_libros_por_categoria(nombre_categoria, url_categoria):
    """Recorre todas las páginas de una categoría y extrae datos clave."""
    libros_lista = []   # Lista para almacenar los libros encontrados
    url_pagina = url_categoria    # URL inicial de la categoría

    while True:
        sopa_pagina = obtener_sopa_html(url_pagina)             # Obtiene el HTML de la página actual
        
        elementos_libros = sopa_pagina.select("article.product_pod h3 a")                 # Selecciona todos los elementos de libro en la página actual
        for libro_elemento in elementos_libros:            # Itera sobre cada libro encontrado
            titulo_libro = libro_elemento.get("title")       # Extrae el título del libro
            url_detalle = urljoin(URL_BASE + "catalogue/", libro_elemento["href"].replace("../", ""))      # Construye la URL completa de la página de detalle del libro
            
            libros_lista.append({        # Agrega un diccionario con los datos clave del libro a la lista
                "titulo": titulo_libro,
                "categoria": nombre_categoria,
                "url": url_detalle
            })

        # Paginación: si hay siguiente página, actualizar url_pagina; si no, terminar
        next_link = sopa_pagina.select_one("li.next a")
        if next_link:
            url_pagina = urljoin(url_pagina, next_link["href"])   # Actualiza la URL para la siguiente página
        else:
            break
        
        time.sleep(0.5)  # Evita sobrecargar el servidor

    return libros_lista


def rasguear_detalles_libro(diccionario_libro):
    """Extrae precio, calificación, stock y autor de la página de detalle."""
    sopa_detalle = obtener_sopa_html(diccionario_libro["url"])     # Obtiene el HTML de la página de detalle del libro
    
    # Precio (limpiando símbolos)
    precio_texto = sopa_detalle.select_one(".price_color").text
    precio_numerico = float(re.sub(r"[^0-9.]", "", precio_texto.replace("Â", "").strip())) # Elimina símbolos y convierte a float
    
    # Calificación   
    calificacion_texto_original = sopa_detalle.select_one(".star-rating")["class"][1]          # Extrae la clase que indica la calificación
    calificacion_numero_convertida = mapeo_calificacion_texto_a_numero.get(calificacion_texto_original, 0)   # Convierte texto a número usando el mapeo
    
    # Stock
    stock_disponible = sopa_detalle.select_one(".availability").text.strip() # Extrae el texto de disponibilidad
    coincidencia_numero_stock = re.search(r'\d+', stock_disponible)    # Busca el número en el texto de disponibilidad
    stock_numero_extraido = int(coincidencia_numero_stock.group()) if coincidencia_numero_stock else 0

   
    nombre_autor = random.choice(AUTORES_FICTICIOS)        # Selecciona un autor ficticio aleatorio

    diccionario_libro.update({   # Actualiza el diccionario del libro con los nuevos datos extraídos
        "precio": precio_numerico,
        "calificacion": calificacion_numero_convertida,
        "stock": stock_numero_extraido,
        "autor_scrapeado": nombre_autor  
    })
    return diccionario_libro


In [ ]:
# =========================
# CELDA 3: Scrapeo completo
# =========================
def obtener_categorias():              # Obtiene todas las categorías disponibles en la página principal
    sopa = obtener_sopa_html(URL_BASE)   # Obtiene el HTML de la página principal
    lista = [] # Lista para almacenar las categorías
    # Ignoramos la primera categoría "Books" que es la página principal
    for cat in sopa.select(".side_categories ul li ul li a"):
        nombre = cat.text.strip()
        url = urljoin(URL_BASE, cat["href"]) # Construye la URL completa de la categoría
        lista.append((nombre, url))
    return lista

todos_libros_scraped = []          #Lista para almacenar todos los libros scrapeados

categorias_disponibles = obtener_categorias()   # Obtiene todas las categorías disponibles
for nombre_cat, url_cat in categorias_disponibles: #Itera sobre cada categoría
    print(f"Scrapeando categoría: {nombre_cat}")     
    libros_categoria_actual = obtener_libros_por_categoria(nombre_cat, url_cat) #   Obtiene todos los libros de la categoría actual
    for libro_dict in libros_categoria_actual:      #Itera sobre cada libro en la categoría actual
        try:
            todos_libros_scraped.append(rasguear_detalles_libro(libro_dict)) # Extrae y agrega los detalles del libro
        except Exception as e: #    Captura errores individuales sin detener todo el proceso
            print(f"Error al scrapear detalles de {libro_dict['url']}: {e}")
        time.sleep(0.2)  # Pequeño delay
        
print(f"Scrapeo completado. Total libros: {len(todos_libros_scraped)}")  


Scrapeando categoría: Travel
Scrapeando categoría: Mystery
Scrapeando categoría: Historical Fiction
Scrapeando categoría: Sequential Art
Scrapeando categoría: Classics
Scrapeando categoría: Philosophy
Scrapeando categoría: Romance
Scrapeando categoría: Womens Fiction
Scrapeando categoría: Fiction
Scrapeando categoría: Childrens
Scrapeando categoría: Religion
Scrapeando categoría: Nonfiction
Scrapeando categoría: Music
Scrapeando categoría: Default
Scrapeando categoría: Science Fiction
Scrapeando categoría: Sports and Games
Scrapeando categoría: Add a comment
Scrapeando categoría: Fantasy
Scrapeando categoría: New Adult
Scrapeando categoría: Young Adult
Scrapeando categoría: Science
Scrapeando categoría: Poetry
Scrapeando categoría: Paranormal
Scrapeando categoría: Art
Scrapeando categoría: Psychology
Scrapeando categoría: Autobiography
Scrapeando categoría: Parenting
Scrapeando categoría: Adult Fiction
Scrapeando categoría: Humor
Scrapeando categoría: Horror
Scrapeando categoría: Histo

In [ ]:
with open("datos_de_libros.json", "w", encoding="utf-8") as archivo_json: # Guarda los datos scrapeados en un archivo JSON
    json.dump(todos_libros_scraped, archivo_json, indent=2, ensure_ascii=False)       # Indenta el JSON para mejor legibilidad

In [ ]:
# =========================
# CELDA 4: Crear base de datos SQLite (DDL)
# =========================
NOMBRE_BD = "Libros.db"
conexion_base_datos = sqlite3.connect(NOMBRE_BD)  # Conecta (o crea) la base de datos SQLite
cursor_operaciones = conexion_base_datos.cursor()   # Crea un cursor para ejecutar comandos SQL

cursor_operaciones.execute("""
CREATE TABLE IF NOT EXISTS Autores (                     
    id INTEGER PRIMARY KEY AUTOINCREMENT,             # Identificador único para cada autor
    nombre TEXT UNIQUE                                  # Nombre del autor único
)
""")

cursor_operaciones.execute("""
CREATE TABLE IF NOT EXISTS Categorias (
    id INTEGER PRIMARY KEY AUTOINCREMENT,                  # Identificador único para cada categoría
    nombre TEXT UNIQUE                                         # Nombre de la categoría único
)
""")

cursor_operaciones.execute("""
CREATE TABLE IF NOT EXISTS Libros (
    id INTEGER PRIMARY KEY AUTOINCREMENT,               # Identificador único para cada libro
    titulo TEXT,                                          # Título del libro
    precio REAL,                                            # Precio del libro
    calificacion INTEGER,                                  # Calificación del libro
    stock INTEGER,                                          # Stock disponible del libro
    id_categoria INTEGER,                                      # Identificador de la categoría del libro
    FOREIGN KEY(id_categoria) REFERENCES Categorias(id)              # Llave foránea a la tabla Categorias
)
""")

cursor_operaciones.execute("""
CREATE TABLE IF NOT EXISTS Libro_Autor (
    id_libro INTEGER,                                              ## Identificador del libro
    id_autor INTEGER,                                             ## Identificador del autor
    PRIMARY KEY (id_libro, id_autor),                                      # Llave primaria compuesta
    FOREIGN KEY (id_libro) REFERENCES Libros(id),                         # Llave foránea a la tabla Libros
    FOREIGN KEY (id_autor) REFERENCES Autores(id)                           # Llave foránea a la tabla Autores
)
""")

conexion_base_datos.commit()                            # Guarda los cambios y crea las tablas
print(f"Esquema de base de datos '{NOMBRE_BD}' creado con estructura M2M.")

Esquema de base de datos 'Libros.db' creado con estructura M2M.


In [ ]:
# =========================
# CELDA 5: Insertar datos en DB
# ========================= 
with open("datos_de_libros.json", "r", encoding="utf-8") as archivo_json:   # Abre el archivo JSON con los datos scrapeados
    lista_datos_libros = json.load(archivo_json)          # Carga los datos en una lista de diccionarios

# Insertar categorías
categorias_a_insertar = list(set([item["categoria"] # Extrae categorías únicas y la convierte en una lista , para luego iterar e insertarlas
for item in lista_datos_libros 
if "categoria" in item]))  #se corrobora que la categoría exista
for cat_nombre in categorias_a_insertar: # Inserta cada categoría en la base de datos
    cursor_operaciones.execute("INSERT OR IGNORE INTO Categorias(nombre) VALUES(?)", (cat_nombre,)) # Inserta la categoría si no existe
conexion_base_datos.commit()
print(f"Categorías insertadas: {len(categorias_a_insertar)} únicas.") # Insertar autores

# Insertar autores
autores_reales = [item["autor_scrapeado"]  # Extrae autores reales
for item in lista_datos_libros  # Itera sobre cada libro
if "autor_scrapeado" in item]       # Filtra solo los libros con autor_scrapeado
autores_a_insertar = list(set(autores_reales + AUTORES_FICTICIOS)) # Combina autores reales y ficticios
for autor in autores_a_insertar:     # Inserta cada autor en la base de datos
    cursor_operaciones.execute("INSERT OR IGNORE INTO Autores(nombre) VALUES(?)", (autor,)) # Inserta el autor si no existe
conexion_base_datos.commit()

# IDs de autores ficticios para llenar la tabla intermedia , o  de relacion muchos a muchos
placeholders = ', '.join(['?'] * len(AUTORES_FICTICIOS))   # placeholders es un marcador de posición para la consulta SQL , llena de signos de interrogación según el número de autores ficticios
query_ids_ficticios = f"SELECT id FROM Autores WHERE nombre IN ({placeholders})"      #El formulario ya está hecho, los datos se llenan después.
cursor_operaciones.execute(query_ids_ficticios, AUTORES_FICTICIOS)   # Ejecuta la consulta para obtener los IDs
ids_autores_ficticios = [row[0] for row in cursor_operaciones.fetchall()] # Lista de IDs de autores ficticios ,Obtengo los IDs para saber qué autor es quién
# el fetchall() me devuelve una lista de tuplas, cada tupla contiene un solo elemento (el ID del autor), por lo que uso row[0] para extraer el ID de cada tupla y crear una lista de IDs.
# Insertar libros y relaciones M2M
libros_insertados_count = 0
for libro_item in lista_datos_libros:   # Itera sobre cada libro en los datos scrapeados
    if "categoria" not in libro_item or "autor_scrapeado" not in libro_item:    # Verifica que el libro tenga categoría y autor
        continue

    cursor_operaciones.execute("SELECT id FROM Categorias WHERE nombre=?", (libro_item["categoria"],))  # Obtiene el ID de la categoría
    id_categoria_obtenida = cursor_operaciones.fetchone()[0]  #fetchone() me retorna una tupla con una sola columna, y con [0] extraigo el valor del ID para usarlo 
    
    cursor_operaciones.execute("SELECT id FROM Autores WHERE nombre=?", (libro_item["autor_scrapeado"],)) 
    id_autor_principal = cursor_operaciones.fetchone()[0] # Obtiene el ID del autor principal

    cursor_operaciones.execute("""
        INSERT INTO Libros(titulo, precio, calificacion, stock, id_categoria) 
        VALUES (?, ?, ?, ?, ?)
    """, (libro_item["titulo"], libro_item["precio"], libro_item["calificacion"], libro_item["stock"], id_categoria_obtenida)) # Inserta el libro en la tabla Libros, con sus datos correspondientes
    
    id_libro_obtenido = cursor_operaciones.lastrowid # Obtiene el ID del libro recién insertado

    # Autor principal
    cursor_operaciones.execute("""              # Inserta la relación libro-autor principal
        INSERT OR IGNORE INTO Libro_Autor(id_libro, id_autor) VALUES (?, ?)
    """, (id_libro_obtenido, id_autor_principal))         # Inserta la relación libro-autor principal

    # Relleno de la relacion de muchos a muchpos: 30% de probabilidad
    if random.random() < 0.3:              # 30% de probabilidad
        num_autores_extra = random.randint(1, 2)         # Número aleatorio de autores extra (1 o 2)
        autores_extra = random.sample(ids_autores_ficticios, num_autores_extra)         #sample garantiza que no se repitan , los ids de los autores ficticios
        for id_extra in autores_extra: 
            if id_extra != id_autor_principal: # Evita duplicados
                cursor_operaciones.execute("""
                    INSERT OR IGNORE INTO Libro_Autor(id_libro, id_autor) VALUES (?, ?)
                """, (id_libro_obtenido, id_extra)) # Inserta la relación libro-autor extra
    
    libros_insertados_count += 1

conexion_base_datos.commit()         # Guarda todos los cambios en la base de datos
print(f"Base de datos poblada con {libros_insertados_count} libros y relaciones M2M simuladas.")







Categorías insertadas: 50 únicas.
Base de datos poblada con 1000 libros y relaciones M2M simuladas.


In [7]:
import sqlite3

conexion = sqlite3.connect("libros.db")
cursor = conexion.cursor()

query = """
SELECT a.nombre AS autor, l.titulo AS libro, l.calificacion
FROM Libros l
JOIN Libro_Autor la ON l.id = la.id_libro
JOIN Autores a ON la.id_autor = a.id
WHERE l.calificacion = 1
ORDER BY a.nombre;
"""

cursor.execute(query)
resultados = cursor.fetchall()

# Mostrar resultados sin que aparezca "Add a comment ->"
for i, (autor, libro, calificacion) in enumerate(resultados, 1):
    print(f"{i}. {autor} -> {libro} ({calificacion} estrella)")

conexion.close()


1. Ana Quiñónez -> The Purest Hook (Second Circle Tattoos #3) (1 estrella)
2. Ana Quiñónez -> Mr. Mercedes (Bill Hodges Trilogy #1) (1 estrella)
3. Ana Quiñónez -> Mesaerion: The Best Science Fiction Stories 1800-1849 (1 estrella)
4. Ana Quiñónez -> The Dovekeepers (1 estrella)
5. Ana Quiñónez -> Sarah's Key (1 estrella)
6. Ana Quiñónez -> Hollow City (Miss Peregrineâs Peculiar Children #2) (1 estrella)
7. Ana Quiñónez -> 8 Keys to Mental Health Through Exercise (1 estrella)
8. Ana Quiñónez -> The E-Myth Revisited: Why Most Small Businesses Don't Work and What to Do About It (1 estrella)
9. Andrés Maldonado -> Algorithms to Live By: The Computer Science of Human Decisions (1 estrella)
10. Andrés Maldonado -> Clockwork Angel (The Infernal Devices #1) (1 estrella)
11. Andrés Maldonado -> Quiet: The Power of Introverts in a World That Can't Stop Talking (1 estrella)
12. Andrés Maldonado -> Island of Dragons (Unwanteds #7) (1 estrella)
13. Andrés Maldonado -> The Long Shadow of Small Gho

In [8]:
import sqlite3

conexion = sqlite3.connect("libros.db")
cursor = conexion.cursor()

# Consulta: libros con más de 3 estrellas y precio < 10
query = """
SELECT titulo, precio, calificacion
FROM Libros
WHERE calificacion > 3 AND precio < 10
ORDER BY calificacion DESC, precio ASC;
"""

cursor.execute(query)
resultados = cursor.fetchall()

for i, fila in enumerate(resultados, 1):
    print(f"{i}. {fila[0]} -> £{fila[1]} ({fila[2]} estrellas)")

conexion.close()


In [ ]:
conexion = sqlite3.connect("libros.db")
cursor = conexion.cursor()

query = """
SELECT a.nombre, COUNT(la.id_libro) AS total_libros
FROM Autores a
JOIN Libro_Autor la ON a.id = la.id_autor
GROUP BY a.nombre
ORDER BY total_libros DESC;
"""

cursor.execute(query)
resultados = cursor.fetchall()

for autor, total in resultados:
    print(f"{autor} -> {total} libros")

conexion.close()


Martín López -> 53 libros
Nadia Paredes -> 50 libros
Camila Duarte -> 49 libros
Ava Lunam -> 48 libros
Sebastián Roldán -> 47 libros
Oscar Lopez -> 46 libros
Valentina Coronel -> 45 libros
Leticia Moreno -> 45 libros
Bruno Acosta -> 45 libros
Nadia Volkov -> 42 libros
Daniela Ortiz -> 42 libros
Kaelen Rhys -> 41 libros
Gustavo Cabrera -> 41 libros
Seraphina Key -> 39 libros
Paula Sosa -> 39 libros
Carla Medina -> 39 libros
Rodrigo Vera -> 38 libros
Sofía Rivas -> 37 libros
Quinn Sparrow -> 37 libros
Laura Benítez -> 37 libros
Ana Quiñónez -> 37 libros
Viola Frost -> 36 libros
Tomás Ferreira -> 36 libros
Ricardo Vázquez -> 36 libros
Zane O’Connell -> 35 libros
Talon Cross -> 35 libros
juana de armas -> 34 libros
Gabriela Ayala -> 34 libros
Federico Román -> 34 libros
Elena Caballero -> 34 libros
Diego Montenegro -> 33 libros
Lucas Giménez -> 32 libros
jose perez -> 31 libros
Julián Barrios -> 30 libros
María Villalba -> 29 libros
Jaxon Bellwether -> 29 libros
Andrés Maldonado -> 21 libr

In [10]:
conexion = sqlite3.connect("libros.db")
cursor = conexion.cursor()

query = """
SELECT c.nombre AS categoria, AVG(l.calificacion) AS promedio
FROM Categorias c
JOIN Libros l ON c.id = l.id_categoria
GROUP BY c.nombre
ORDER BY promedio DESC;
"""

cursor.execute(query)
resultados = cursor.fetchall()
for i, fila in enumerate(resultados, 1):
    print(f"{i}. {fila[0]} -> promedio {fila[1]:.2f}")
conexion.close()


1. Novels -> promedio 5.00
2. Erotica -> promedio 5.00
3. Adult Fiction -> promedio 5.00
4. Christian Fiction -> promedio 4.17
5. Health -> promedio 3.75
6. Art -> promedio 3.62
7. Poetry -> promedio 3.53
8. Humor -> promedio 3.40
9. Spirituality -> promedio 3.33
10. Young Adult -> promedio 3.30
11. Historical Fiction -> promedio 3.23
12. Fiction -> promedio 3.18
13. New Adult -> promedio 3.17
14. Music -> promedio 3.15
15. Religion -> promedio 3.14
16. Womens Fiction -> promedio 3.12
17. Fantasy -> promedio 3.08
18. Suspense -> promedio 3.00
19. Sports and Games -> promedio 3.00
20. Historical -> promedio 3.00
21. Autobiography -> promedio 3.00
22. Sequential Art -> promedio 2.97
23. History -> promedio 2.94
24. Mystery -> promedio 2.94
25. Science -> promedio 2.93
26. Business -> promedio 2.92
27. Food and Drink -> promedio 2.90
28. Nonfiction -> promedio 2.88
29. Default -> promedio 2.84
30. Add a comment -> promedio 2.76
31. Travel -> promedio 2.73
32. Thriller -> promedio 2.73
33.

In [2]:
import time
conexion = sqlite3.connect("libros.db")

cursor = conexion.cursor()

# ---- Sin índice ----
inicio = time.time()
cursor.execute("SELECT * FROM Libros WHERE calificacion = 5;")
cursor.fetchall()
fin = time.time()
print(f"Tiempo sin índice: {fin - inicio:.5f} segundos")

# ---- Crear índice ----
cursor.execute("CREATE INDEX IF NOT EXISTS idx_calificacion ON Libros(calificacion);")
conexion.commit()                         #índice de materias, que permite saltar directo a lo que te interesa sin leer todo

# ---- Con índice ----
inicio = time.time()
cursor.execute("SELECT * FROM Libros WHERE calificacion = 5;")
cursor.fetchall()
fin = time.time()
print(f"Tiempo con índice: {fin - inicio:.5f} segundos")

conexion.close()


NameError: name 'sqlite3' is not defined